In [323]:
import requests
import pandas as pd
import numpy as np
import sys
from importlib import reload
import os.path
from os import path
import sys
import math

In [29]:
sys.path.append('..') #agregar directorio madre del repo para poder importar librerias customizadas
#importar librerias customizadas

In [30]:
import modules.html_process
reload(modules.html_process)

<module 'modules.html_process' from '../modules/html_process.py'>

In [31]:
from modules.html_process import *

In [38]:
url='https://www.chileatiende.gob.cl/api/fichas?access_token=q1bYCHOyXMOoAD6S&query=&maxResults=100&pageToken='

In [39]:
fichas=[]
for p in range(1,27):
    f=requests.get(url+str(p))
    f=f.json()
    f=f['fichas']['items']['ficha']
    for i in f:
        fichas.append(i)

In [40]:
len(fichas)

2537

In [96]:
fichascha=pd.DataFrame(fichas) #convertir las fichas a dataframe pandas
fichascha['id']=fichascha['id'].astype('object') #convertir campo id a tipo 'object'

In [97]:
fichascha=html_process(fichascha) #convertirla a objeto 'html_process' para borrar tags html de campos string

In [98]:
fichascha.fields=['objetivo','detalle','beneficiarios','costo','doc_requeridos','vigencia','guia_oficina',
                 'guia_online','guia_online_url','guia_oficina','guia_correo','guia_telefonico','guia_consulado'
                 
                 ] #setear campos en los cuales borrar html tags

In [99]:
fichascha.remove_html_tags_pattern() #ejecutar función de limpieza de tags html en campos string

In [100]:
fichascha=fichascha.df #devolver resultados a df original

In [101]:
#cambios de nombre de campos para estandarizar con RNT

rnt_mapping={'permalink':'url_chileatiende', #url de la ficha chileatiende
 'marco_legal':'url_marco_legal', #url marco legal
 'guia_online_url': 'url_guia_institucional', #url descripción institucional

}

In [102]:
fichascha=fichascha.rename(columns=rnt_mapping)

In [103]:
#flag de si tiene atención presencial---->para homologar con RNT
fichascha['atencion_presencial']='No' #asumir a priori todos valores 'No', luego cambiar con operaciones de abajo a 'Si' casos que corresponde
fichascha.loc[fichascha['guia_oficina'].str.replace(' ','').str.len()>0,'guia_oficina']='Si' #registros que no están blanco
fichascha.loc[fichascha['guia_oficina'].notnull(),'atencion_presencial']='Si' #registros que no son nulos

In [104]:
#flag de si tiene atención telefonico---->para homologar con RNT
fichascha['atencion_telefonico']='No' #asumir a priori todos valores 'No', luego cambiar con operaciones de abajo a 'Si' casos que corresponde
fichascha.loc[fichascha['guia_telefonico'].str.replace(' ','').str.len()>0,'guia_telefonico']='Si' #registros que no están blanco
fichascha.loc[fichascha['guia_telefonico'].notnull(),'atencion_telefonico']='Si' #registros que no son nulos

In [105]:
#flag de si tiene atención digital (asumido a partir de guia online url)---->para homologar con RNT
fichascha['atencion_digital']='No' #asumir a priori todos valores 'No', luego cambiar con operaciones de abajo a 'Si' casos que corresponde
fichascha.loc[fichascha['guia_online'].str.replace(' ','').str.len()>0,'guia_online']='Si' #registros que no están blanco
fichascha.loc[fichascha['guia_online'].notnull(),'atencion_digital']='Si' #registros que no son nulos

In [106]:
#flag de fichas que no tienen costo (gratis)---->para homologar con RNT
#descripción si tiene costo , valores ['No', 'Si', 'No informa', 'En algunos casos']
fichascha['Pago']='No informa' #valor por defecto
fichascha.loc[fichascha['costo'].str.contains('No tiene costo|no tiene costo'),'Pago']='No' #string 1 O ('|') string 2--->No tiene costo

In [107]:
#Incluir info si tiene pago
fichascha.loc[(fichascha['costo'].str.replace(' ','').str.len()>0) & (fichascha['Pago']!='No'),'Pago']='Si' #Tiene costo

In [108]:
#replace_values = {'No tiene costo' : '', 'no tiene costo' : '' , '.' : '',',':'',' ':'','\xa0':''}   

In [109]:
#fichascha.replace({'costo':replace_values})['costo'].unique()

In [110]:
#fichascha['costo'][(fichascha['Pago']=='No') & fichascha['costo'].str.replace('No tiene costo|no tiene costo','')].unique()

In [111]:
fichascha.sample(10)

,id,codigo,servicio,fecha,titulo,objetivo,detalle,beneficiarios,costo,url_marco_legal,...,updated_data_at,observaciones,temas,tags,chileclic_id,url_chileatiende,atencion_presencial,atencion_telefonico,atencion_digital,Pago
254,10978,AB006-8,Servicio Electoral,2020-01-02 00:00:00,Centro de documentación y normativa electoral,Permite acceder a textos archivados en la bibl...,,Todas las personas.,Sólo costo de reproducción de lo requerido (fo...,,...,2020-01-02 00:00:00,,{'tema': []},[],None,https://www.chileatiende.gob.cl/fichas/10978-c...,Si,Si,Si,Si
1635,56619,AD022-56619,Subsecretaría para las Fuerzas Armadas,2020-01-10 00:00:00,Otorgamiento o modificación sustancial de conc...,Permite solicitar el otorgamiento de una conce...,Se refiere a bienes nacionales de uso público ...,"Para el otorgamiento, personas naturales o ju...","No tiene costo. Sin embargo, en caso de otorga...","<p><a style=""background-color: #ffffff;"" href=...",...,2020-01-10 00:00:00,,{'tema': []},[Permiso Sectorial],None,https://www.chileatiende.gob.cl/fichas/56619-o...,Si,Si,Si,No
2452,60851,AP017-60851,Parque Metropolitano de Santiago,2020-03-17 00:00:00,Horarios y tarifas de atractivos del Parquemet,Permite conocer los horarios y tarifas de los ...,,Todas las personas.,No tiene costo.,,...,2020-03-17 00:00:00,,{'tema': []},[],None,https://www.chileatiende.gob.cl/fichas/60851-h...,Si,Si,Si,No
2336,745,ZC005-9,Corporación Nacional Forestal,2020-01-07 00:00:00,Filmaciones o fotografías en áreas silvestres ...,Permite obtener un permiso para realizar filma...,,"Audiovisualistas, cineastas, realizadores o re...",Comerciales o filmaciones publicitarias: 50 U...,"<p><a href=""http://www.conaf.cl/cms/editorweb/...",...,2020-01-07 00:00:00,,{'tema': []},[],None,https://www.chileatiende.gob.cl/fichas/745-fil...,Si,Si,Si,Si
2457,5194,AL005-23,Instituto de Previsión Social,2020-02-18 00:00:00,Bono por Hijo,Es un beneficio que incrementa el monto de la ...,,Madres afiliadas a una AFP que obtuvieron su ...,No tiene costo.,"<p><a href=""http://www.leychile.cl/Navegar?idN...",...,2020-02-18 00:00:00,,{'tema': []},"[Familia, Previsión, Bonos, Mujer]",None,https://www.chileatiende.gob.cl/fichas/5194-bo...,Si,Si,Si,No
1350,328,AK008-14,Corporación de Asistencia Judicial Región Metr...,2020-01-31 00:00:00,Asesoría jurídica para determinar el régimen d...,Permite solicitar orientación y asesoría juríd...,La determinación de un régimen de relación dir...,A todos los padres y madres que no ejercen el ...,"Para orientación e información, la Corporación...",<ul> <li>Artículos 229 y siguientes del <a hre...,...,2020-01-31 00:00:00,,{'tema': []},[Legal],None,https://www.chileatiende.gob.cl/fichas/328-ase...,Si,Si,Si,Si
2318,4666,AM011-4,Superintendencia de Servicios Sanitarios,2020-01-10 00:00:00,Renovación de la credencial de instaladores sa...,Permite solicitar la renovación de la credenci...,,Sólo instaladores sanitarios no profesionales ...,No tiene costo.,"<ul> <li><a href=""http://www.siss.gob.cl/appsi...",...,2020-01-10 00:00:00,,{'tema': []},[Trabajo],None,https://www.chileatiende.gob.cl/fichas/4666-re...,Si,Si,Si,No
2517,33958,ZZ001-221,ChileAtiende,2020-05-19 00:00:00,Pensión Básica Solidaria (PBS),Es un beneficio monetario mensual al que puede...,,,,,...,2020-05-19 00:00:00,,{'tema': []},[Previsión],None,https://www.chileatiende.gob.cl/fichas/33958-p...,Si,Si,Si,No informa
700,13438,AE006-113,Servicio de Impuestos Internos,2020-01-07 00:00:00,Tasación fiscal de vehículos,Permite conocer el valor de la tasación fiscal...,,Personas naturales y jurídicas.,No tiene costo.,"<ul> <li><a href=""http://www.sii.cl/normativa_...",...,2020-01-07 00:00:00,,{'tema': []},[Legal],None,https://www.chileatiende.gob.cl/fichas/13438-c...,Si,Si,Si,No
1198,23037,AI000-12,Ministerio de Desarrollo Social y Familia,2020-01-02 00:00:00,Programa Noche Digna,Entrega alternativas de alojamiento temporal y...,,Personas en situación de calle.,No tiene costo.,,...,2020-01-02 00:00:00,,{'tema': []},

In [301]:
import modules.Elastic_Module
reload(modules.Elastic_Module)
from modules.Elastic_Module import *

In [302]:
#variables de entorno
args={}
args['elastic_user']='elastic'
args['elastic_psswd']=os.environ['ELASTIC_PASSWD']

In [303]:
ES_HOST = "https://"
basic_auth={'user':args['elastic_user'], 'psswd':args['elastic_psswd']}

In [304]:
es=elastic_connection(ES_HOST,basic_auth=basic_auth)

In [305]:
es.ping()

True


In [306]:
indexes=[
    ('cha','cha')
        ]

In [307]:
#generateID_params={'x':'id','y':'_id'} #cambiar nombres de campos (parámetros)

In [308]:
#generar campos concatenados (parámetros)
#generateID_params_custom={'fields':['package_id','id'], #campos a concatenar
#                               'head':20, #numero de caracteres a dejar
#                                'sep':'_' #separador entre campos
#                               }

In [309]:
#es.generateID_params_custom=generateID_params_custom

In [310]:
#es.generateID_params=generateID_params

In [311]:
es.add_docs(fichascha)

In [312]:
#prepar la data para elastic search
es.elastic_prepare_pattern()

In [313]:
i=indexes[0]
n=1500
id_field='id'
es.bulk_data(INDEX_NAME=i[0],_type=i[1],n=n,id_field=id_field)
#test

creando paquetes de data bulk
size of bulk docs is : 0.1015625 Kbs
N° docs: 2537.0, N° chunks: 4


In [314]:
len(es.chunks[0])

1500

In [315]:
es.clear_index(i[0])

deleting 'cha' index...
 response: '{'acknowledged': True}'
creating 'cha' index...
 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'cha'}'


In [316]:
es.bulk_index()

bulk indexing...:index---> cha
 indexing chunk : 3 of 4finished bulk indexing
No recorded ES indexing errors
No recorded ES indexing exceptions


In [317]:
es.test_index(i[0],n=15)

../modules/Elastic_Module.py:78: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df = json_normalize([x['_source'] for x in res['hits']['hits']])


,codigo,servicio,fecha,titulo,objetivo,detalle,beneficiarios,costo,url_marco_legal,guia_online,...,updated_data_at,observaciones,tags,chileclic_id,url_chileatiende,atencion_presencial,atencion_telefonico,atencion_digital,Pago,temas.tema
0,AL005-31,Instituto de Previsión Social,2020-05-04 00:00:00,Fecha y forma de pago de una pensión u otros b...,Permite conocer el día y la forma en que el IP...,Información importante sobre el pago de benefi...,Pensionados y pensionadas de las ex cajas de p...,No tiene costo.,,Si,...,2020-05-04 00:00:00,,[Previsión],None,https://www.chileatiende.gob.cl/fichas/5212-fe...,Si,Si,Si,No,[]
1,AL005-51,Instituto de Previsión Social,2020-05-04 00:00:00,Modificación de forma de pago para beneficiari...,Permite a personas que reciben algún beneficio...,{{youtube:Jz6JLTIwbS8}},Pensionados y pensionadas de las ex cajas de p...,No tiene costo.,,Si,...,2020-05-04 00:00:00,,[Previsión],None,https://www.chileatiende.gob.cl/fichas/5268-mo...,Si,Si,Si,No,[]
2,AL005-221,Instituto de Previsión Social,2020-05-04 00:00:00,Bono de Invierno 2020,Es un beneficio no postulable de 64 mil 549 pe...,,Los beneficiarios y beneficiarias deben cumpli...,No tiene costo.,"<p><a href=""http://bcn.cl/2878a"" target=""_blan...",,...,2020-05-04 00:00:00,,[Previsión],None,https://www.chileatiende.gob.cl/fichas/39484-b...,Si,Si,Si,No,[]
3,ZZ001-66824,ChileAtiende,2020-05-20 00:00:00,Comisaría Virtual,"Es una plataforma web de Carabineros de Chile,...",,,,,,...,2020-05-20 00:00:00,,[],None,https://www.chileatiende.gob.cl/fichas/66824-c...,Si,Si,Si,No informa,[]
4,ZZ001-78383,ChileAtiende,2020-04-24 00:00:00,Protocolos sanitarios para el funcionamiento d...,En el marco de la crisis sanitaria por el Coro...,,,,,,...,2020-04-24 00:00:00,,[],None,https://www.chileatiende.gob.cl/fichas/78383-p...,Si,Si,Si,No informa,[]
5,AI000-77717,Ministerio de Desarrollo Social y Familia,2020-05-08 00:00:00,Postulación al Ingreso Mínimo Garantizado (IMG),Permite postular al Ingreso Mínimo Garantizado...,,A todos los trabajadores y trabajadoras depend...,No tiene costo.,"<p><a href=""https://www.leychile.cl/Navegar?id...",Si,...,2020-05-08 00:00:00,,[Trabajo],None,https://www.chileatiende.gob.cl/fichas/77717-p...,Si,Si,Si,No,[]
6,ZZ001-151,ChileAtiende,2020-05-18 00:00:00,Subsidio Familiar (SUF),También conocido como Subsidio Único Familiar ...,Importante: El subsidio familiar de personas ...,Son beneficiarias aquellas personas que no est...,No tiene costo.,"<ul> <li><a href=""http://bcn.cl/27276"" target=...",,...,2020-05-18 00:00:00,,"[Salud, Familia, Previsión, Subsidios]",None,https://www.chileatiende.gob.cl/fichas/33112-s...,Si,Si,Si,No,[]
7,AI000-55039,Ministerio de Desarrollo Social y Familia,2020-05-19 00:00:00,Solicitud de rectificación de datos administra...,Permite solicitar la corrección de información...,,Personas mayores de edad que sean integrantes ...,No tiene costo.,"<ul> <li><a href=""https://www.leychile.cl/Nave...",,...,2020-05-19 00:00:00,,[],None,https://www.chileatiende.gob.cl/fichas/55039-s...,Si,Si,Si,No,[]
8,ZZ001-77485,ChileAtiende,2020-05-20 00:00:00,Permiso temporal para salir en cuarentena,Permite a las personas acceder al permiso temp...,,"A quienes deban: Asistir a centros, servicios...",No tiene costo.,,Si,...,2020-05-20 00:00:00,,[],None,https://www.chileatiende.gob.cl/fichas/77485-p...,Si,Si,Si,No,[]
9,AL005-72,Instituto de Previsión Social,2020-04-06 00:00:00,Autorización para cobrar una Pensión Básica So...,Permite a un tercero recibir autorización de u...,Formas en que esta autorización se puede desig...,Pensionados o pensionadas de una Pensión Básic...,No tiene costo.,"<p><a href=""http://www.leychile.cl/Navegar?idN...",Si,...,2020-04-06 00:00:00,,[Previsión],None,https://www.chileatiende.gob.cl/fichas/5422-au...,Si,Si,Si,No,[]


In [318]:
es.count_documents(i[0])

[{'epoch': '1590127133', 'timestamp': '05:58:53', 'count': '2537'}]
